### dropbox

In [1]:
# import dropbox
# from dropbox import DropboxOAuth2FlowNoRedirect

# APP_KEY = "qi3onmgezlr00yv"
# auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY, use_pkce=True, token_access_type='offline')

# authorize_url = auth_flow.start()
# print("1. Go to: " + authorize_url)
# print("2. Click \"Allow\" (you might have to log in first).")
# print("3. Copy the authorization code.")
# auth_code = input("Enter the authorization code here: ").strip()

# try:
#     oauth_result = auth_flow.finish(auth_code)
# except Exception as e:
#     print('Error: %s' % (e,))
#     exit(1)

# with dropbox.Dropbox(oauth2_refresh_token=oauth_result.refresh_token, app_key=APP_KEY) as dbx:
#     dbx.users_get_current_account()
#     print("Successfully set up client!")

In [3]:
# print(oauth_result.refresh_token)

b_N3-an_VvQAAAAAAAAAAc7kTAMBxeYC9GlsLpwgFrXX3kbZr2li3f3GfGx-iGnr


##

In [1]:
import dropbox
import os
from io import BytesIO
import time
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

APP_KEY = 'qi3onmgezlr00yv'
REFRESH_TOKEN = 'b_N3-an_VvQAAAAAAAAAAc7kTAMBxeYC9GlsLpwgFrXX3kbZr2li3f3GfGx-iGnr'

In [2]:
def rss_coil_combine(image):
    squared_abs = np.abs(image)**2
    sum_of_squares = np.sum(squared_abs, axis=1)
    combined_images = np.sqrt(sum_of_squares)
    return combined_images

def normal_in_i(image):
    mean = np.mean(image)
    std = np.std(image)
    tmp = (image - mean) / (std + 1e-10)
    return tmp

def extract_subject_id(filename):
    numbers = re.findall(r'\d+', filename)
    if numbers:
        return int(numbers[-1])
    return None

def ifft2c(k):
    x = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(k, (-2,-1)), norm='ortho'), (-2,-1))
    return x

def fft2c(img):
    k = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(img, (-2,-1)), norm='ortho'), (-2,-1))
    return k

In [ ]:
def combine_dict(clear_dict, motion_dict):
    combined_dict = {}

    for key in clear_dict.keys() | motion_dict.keys(): 
        combined_dict[key] = {
            "clear_volume": clear_dict.get(key),
            "motion_volume": motion_dict.get(key)
        }
    return combine_dict

In [ ]:
def crop(img, crop_size=320):
    height, width = img.shape[-2:]
    w_from = (width - crop_size) // 2
    w_to = w_from + crop_size
    h_from = (height - crop_size) // 2
    h_to = h_from + crop_size

    return img[:, h_from:h_to, w_from:w_to]

def padding(volume, size=320):
    slices, height, width = volume.shape
    pad_width = max(0, size - width) // 2 

    padded_volume = np.pad(
        volume,
        pad_width=((0, 0), (0, 0), (pad_width, pad_width)), 
        mode='constant',
        constant_values=0
    )
    return padded_volume

In [3]:
def dropbox_connect():
    dbx = dropbox.Dropbox(oauth2_refresh_token=REFRESH_TOKEN, app_key=APP_KEY)
    return dbx

def download_npy_file(dbx, dropbox_path, max_retries=3, delay=5):
    for attempt in range(max_retries):
        try:
            print(f"Downloading {dropbox_path}, attempt {attempt + 1}")
            _, response = dbx.files_download(dropbox_path)
            file_data = response.content
            npy_data = np.load(BytesIO(file_data))
            return npy_data
        except Exception as e:
            print(f"Error downloading {dropbox_path}: {e}")
            if attempt + 1 == max_retries:
                print(f"Max retries reached for {dropbox_path}")
                return None
            else:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)


def load_npy_from_dropbox(dbx, folder_path, max_workers=12):
    data_dict = {}

    try:
        folder_metadata = dbx.files_list_folder(folder_path)
        futures = {}

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for entry in folder_metadata.entries:
                if isinstance(entry, dropbox.files.FileMetadata) and entry.name.endswith('.npy'):
                    file_path = os.path.join(folder_path, entry.name)
                    subject_id = extract_subject_id(entry.name)
                    future = executor.submit(download_npy_file, dbx, file_path)
                    futures[future] = subject_id
                    
            for future in as_completed(futures):
                subject_id = futures[future]
                volume = future.result()
                if volume is not None:
                    volume = rss_coil_combine(volume)
                    volume = crop(padding(volume,396),396)
                    for i in range(volume.shape[0]):
                        volume[i, :, :] = normal_in_i(volume[i, :, :])
                    data_dict[subject_id] = volume[:8]

    except Exception as e:
        print(f"Error accessing folder {folder_path}: {e}")

    return data_dict

def load_file_from_dropbox(file_path):
    dbx = dropbox_connect()
    try:
        metadata, res = dbx.files_download(file_path)
        file_stream = BytesIO(res.content)
        np_array = np.load(file_stream)
        
        return np_array

    except dropbox.exceptions.ApiError as err:
        print(f'Error: {err}')
        return None

In [5]:
if __name__ == "__main__":
    dbx = dropbox_connect()
    dropbox_folder_path = '/Motion_data_whole_notrans_flair_0922'
    flair_motion = load_npy_from_dropbox(dbx, dropbox_folder_path)
    print(f"Loaded {len(flair_motion)} npy files.")

Error downloading /Motion_data_whole_notrans_flair_0922/brain_AXFLAIR_200_6002579_motion.npy: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
Loaded 163 npy files.


In [ ]:
if __name__ == "__main__":
    dbx = dropbox_connect()
    dropbox_folder_path = '/fastMRI_data_h5_flair'
    flair_clear = load_npy_from_dropbox(dbx, dropbox_folder_path)
    print(f"Loaded {len(flair_clear)} npy files.")

In [15]:
# /Motion_data_whole_notrans_flair_0922/brain_AXFLAIR_200_6002541_motion.npy
# flair[3]

# volume = load_file_from_dropbox('/Motion_data_whole_notrans_flair_0922/brain_AXFLAIR_200_6002541_motion.npy')

In [11]:
# output_folder = "/root/motioncorrection/flair"
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# n=0
# for volume in flair:
#     n += 1
#     print(volume.shape)
#     plt.figure(figsize=(20, 10))  
#     for i in range(len(volume)):
#         plt.subplot(2, 8, i+1)
#         plt.imshow(volume[i], cmap='gray', origin='lower')
#         plt.axis('off')
#     plt.tight_layout()
#     output_path = os.path.join(output_folder, f"img_{n}.png")
#     plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(14, 640, 264)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 512, 213)
(16, 512, 234)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 512, 276)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 264)
(16, 640, 264)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 512, 276)
(14, 512, 276)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 512, 276)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 320)
(16, 512, 276)
(16, 640, 320)
(16, 512, 276)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 640, 320)
(16, 640, 264)
(16, 512, 276)
(14, 640, 320)
(16, 640, 312)
(16, 512, 320)
(16, 512, 276)
(16, 640, 320)
(16, 640, 320)
(16, 640, 264)
(16, 512, 276)
(16, 640, 

In [6]:
if __name__ == "__main__":
    dbx = dropbox_connect()
    dropbox_folder_path = '/Motion_data_whole_notrans_t2_0922'
    t2_motion = load_npy_from_dropbox(dbx, dropbox_folder_path)
    print(f"Loaded {len(t2_motion)} npy files.")

Error downloading /Motion_data_whole_notrans_t2_0922/brain_AXT2_207_2070613_motion.npy: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
Error downloading /Motion_data_whole_notrans_t2_0922/brain_AXT2_205_2050141_motion.npy: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
Error downloading /Motion_data_whole_notrans_t2_0922/brain_AXT2_203_2030163_motion.npy: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
Loaded 165 npy files.


In [ ]:
if __name__ == "__main__":
    dbx = dropbox_connect()
    dropbox_folder_path = '/Clear_data_notrans_t2_0922'
    t2_clear = load_npy_from_dropbox(dbx, dropbox_folder_path)
    print(f"Loaded {len(t2_clear)} npy files.")

In [12]:
# output_folder = "/root/motioncorrection/t2"
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# n=0
# for volume in t2:
#     n += 1
#     print(volume.shape)
#     plt.figure(figsize=(20, 10))  
#     for i in range(len(volume)):
#         plt.subplot(2, 8, i+1)
#         plt.imshow(volume[i], cmap='gray', origin='lower')
#         plt.axis('off')
#     plt.tight_layout()
#     output_path = os.path.join(output_folder, f"img_{n}.png")
#     plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

(16, 768, 396)


(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(16, 768, 392)
(16, 768, 396)
(14, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(14, 640, 320)
(16, 768, 396)
(16, 768, 392)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 640, 320)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 768, 396)
(16, 768, 396)
(16, 640, 320)
(16, 768, 396)
(16, 640, 320)
(16, 768, 

In [ ]:
# /Motion_data_whole_notrans_t2_0922/brain_AXT2_200_6002490_motion.npy
#  t2[33]

# volume = load_file_from_dropbox('/Motion_data_whole_notrans_t2_0922/brain_AXT2_200_6002490_motion.npy')

In [7]:
if __name__ == "__main__":
    dbx = dropbox_connect()
    dropbox_folder_path = '/Motion_data_whole_notrans_t1post_0922'
    t1post = load_npy_from_dropbox(dbx, dropbox_folder_path)
    print(f"Loaded {len(t1post)} npy files.")

In [ ]:
if __name__ == "__main__":
    dbx = dropbox_connect()
    dropbox_folder_path = '/Clear_data_notrans_t1post_0922'
    t1post_clear = load_npy_from_dropbox(dbx, dropbox_folder_path)
    print(f"Loaded {len(t1post_clear)} npy files.")

In [19]:
# output_folder = "/root/motioncorrection/t1post"
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# n=0
# for volume in t1post:
#     n += 1
#     print(volume.shape)
#     plt.figure(figsize=(20, 10))  
#     for i in range(len(volume)):
#         plt.subplot(2, 8, i+1)
#         plt.imshow(volume[i], cmap='gray', origin='lower')
#         plt.axis('off')
#     plt.tight_layout()
#     output_path = os.path.join(output_folder, f"img_{n}.png")
#     plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

In [ ]:
t1post = [crop(padding(volume,396),396) for volume in t1post]
t2 = [crop(padding(volume,396),396) for volume in t2]
flair = [crop(padding(volume,396),396) for volume in flair]

print(t1post.shape, t2.shape, flair.shape)

In [ ]:
from skimage.transform import rotate

flair[3] = rotate(flair[3], 270, resize=False)
t2[33] = rotate(t2[33], 270, resize=False)

plt.figure(figsize=(20, 5))  
for i in range(len(flair[3])):
    plt.subplot(2, 8, i+1)
    plt.imshow(flair[3][i], cmap='gray', origin='lower')
    plt.axis('off')
plt.tight_layout()

plt.figure(figsize=(20, 5))  
for i in range(len(t2[33])):
    plt.subplot(2, 8, i+1)
    plt.imshow(t2[33][i], cmap='gray', origin='lower')
    plt.axis('off')
plt.tight_layout()